# Building conversational applications with the Converse API

In this notebook, you learn how to use the flexible Converse API to integrate external capabilities into conversational applications.

Certain conversational applications demand an adaptable sequence of calls to language models and various utilities depending on user input. The Converse API enables building such flexible dialogue agents.

## Environment setup

In this task, you set up your environment.

In [1]:
#create a service client by name using the default session.
import json
import os
import boto3
from langchain_community.tools import DuckDuckGoSearchRun

bedrock_client = boto3.client(
    'bedrock-runtime',
    region_name = os.environ.get("AWS_DEFAULT_REGION", None)
)
model_id = "amazon.nova-lite-v1:0"

## Synergizing Reasoning and Acting in Language Models Framework

In this task, the Converse API enables integrating large language models with external capabilities to obtain additional information that results in more accurate and factual responses in a conversation.

The Converse framework allows conversational models like Nova to generate natural language responses in an ongoing dialogue. Within these responses, you can configure markup like [tools] tags to trigger calls to external capabilities.

Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion.

Producing reasoning explanations enables the models to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the models to interface with and obtain information from external sources such as knowledge bases or environments.

Next, you will build a simple calculator agent to add, subtract, multiply, and divide using the Converse API. The purpose is to demonstrate how to register the calculator functions as conversational capabilities that can be called through the Converse API. 

In [2]:
# write a function to add, subtract, multiply, and divide two numbers

def calculator(oper1, oper2, operator):

    def add(a, b):
        return a+b

    def subtract(a, b):
        return a-b

    def multiply(a, b):
        return a*b

    def divide(a, b):
        if b==0:		
            return "Cannot divide by zero"
        else:
            return float(a/b)
        
    if operator == '+' or operator == 'add':
        return add(oper1, oper2)
    elif operator == '-' or operator == 'subtract':
        return subtract(oper1,oper2)
    elif operator == '*' or operator == 'multiply':
        return multiply(float(oper1), float(oper2))
    elif operator == '/' or operator == 'divide':
        return divide(oper1,oper2)
    else:
        return "Invalid operator"

Next, you extend the conversational assistant with access a search engine like DuckDuckGo.

The intent of the following code is to demonstrate:

- Register it as an additional capability with our Converse API
- Update the prompt templates to invoke this capability based on certain trigger phrases


In [3]:
def duckduckgo_search(query: str):
    tool = DuckDuckGoSearchRun()
    result = tool.invoke(query)
    return result

Now, you define a tool configuration to register external capabilities that can be invoked within conversations with an Amazon Bedrock model.

In [4]:
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "duckduckgo_search",
                "description": "Useful when you need to search the internet and find current information.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "Your search query."
                            }
                        },
                        "required": [
                            "query"
                        ]
                    }
                }
            }
        },
        {
            "toolSpec": {
                "name": "calculator",
                "description": "Useful when you need to answer questions that involve adding, subtracting, multiplying, and dividing two numbers. The division can go up to 15 decimal points.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "oper1": {
                                "type": "integer",
                                "description": "first operand."
                            },
                            "oper2": {
                                "type": "integer",
                                "description": "second operand."
                            },
                            "operator": {
                                "type": "string",
                                "description": "operator."
                            }
                        },
                        "required": [
                            "oper1","oper2","operator"
                        ]
                    }
                }
            }
        }
    ]
}

Next, you create a function to call the converse API to converse with an Amazon Bedrock model to generate text responses.

In [5]:
def generate_text(bedrock_client, model_id, tool_config, message_list):
    
    response = bedrock_client.converse(
        modelId=model_id,
        messages=message_list,
        toolConfig=tool_config,
        system=[{"text": "You are a helpful assistant. You can use tools to help you answer questions. You should always use tools to help you with calculations. You can also use tools to help you search the internet and find current information."}]
    )
    
    return response


Next, you create a function to run the tools that the text generated by the Converse API described implementing the ACT of ReACT. This function is the core logic for the interpretation of the text generation response:
* It loops across all content blocks of the response
* It calls the functions of the tools defined above based on what the model told it to do.
* It appropriately adds the output of the function calls of the tools defined above.

In [ ]:
def run_the_tools(response_message):
    
    response_content_blocks = response_message['content']
    
    follow_up_content_blocks = []
    
    for content_block in response_content_blocks:
        if 'toolUse' in content_block:
            tool = content_block['toolUse']
            
            if tool['name'] == 'duckduckgo_search':
                output_search = duckduckgo_search(tool['input']['query'])
                follow_up_content_blocks.append({
                    "toolResult": {
                        "toolUseId": tool['toolUseId'],
                        "content": [{"json": {"result": output_search}}]
                    }
                })
            elif tool['name'] == 'calculator':
                output_calculator = calculator(tool['input']['oper1'], tool['input']['oper2'], tool['input']['operator'])
                follow_up_content_blocks.append({
                    "toolResult": {
                        "toolUseId": tool['toolUseId'],
                        "content": [{"json": {"result": output_calculator}}]
                    }
                })
     
    if len(follow_up_content_blocks) > 0:
        follow_up_message = {
            "role": "user",
            "content": follow_up_content_blocks,
        }
        
        return follow_up_message
    else:
        return None


The `converse` function run an infinite loop and implements the following logic:
1. Calls the `generate_text` function for the Converse API of Bedrock.
2. Appends the output of the model in the message list as the Converse API will need the context.
2. Calls the `run_the_tools` function to run the tools based on the response of #1.
3. Appends the output of the `run_the_tools` to the message list as the Converse API will need the answer in the context.
4. Decides if it needs to go back to #1 if there was a tool used or end the loop

It finally sends back the list of messages containing the whole conversation.

In [7]:
def converse(prompt, bedrock_client, model_id, tool_config):
    
    message_list = [
        {
            "role": "user",
            "content": [ { "text": prompt } ]
        }
    ]
    
    while True:
        response = generate_text(bedrock_client, model_id, tool_config, message_list)
        
        response_message = response['output']['message']
        message_list.append(response_message)
        
        follow_up_message = run_the_tools(response_message)
        
        if follow_up_message is None:
            # No remaining work to do, return final response to user
            break
        else:
            message_list.append(follow_up_message)
            
    return message_list

Next, you send your query to the `converse` function that orchestrates the end-to-end flow to have a conversation with the Amazon Bedrock model leveraging the integrated tools.

In [8]:
# query = "What is an Astronomical unit (AU)? Express the circumference of the earth in AU"
query = "What is Amazon SageMaker? What is launch year multiplied by 2"

messages = converse(query, bedrock_client, model_id, tool_config)

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=4))


MESSAGES:

[
    {
        "role": "user",
        "content": [
            {
                "text": "What is Amazon SageMaker? What is launch year multiplied by 2"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "<thinking> I need to first define what Amazon SageMaker is and then calculate the launch year multiplied by 2. I will use a search tool to find the launch year of Amazon SageMaker and then a calculator tool to perform the multiplication. </thinking>\n"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_YozOF4wRT3KM6G5_mJD4Ww",
                    "name": "duckduckgo_search",
                    "input": {
                        "query": "What is Amazon SageMaker?"
                    }
                }
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_TwFg0E4GRV2_bw21-6IKrg",
                    "na

You have successfully integrated Amazon Bedrock with custom capabilities by utilizing Converse API and tool framework.